# Model definition (for hap-fr)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.convolution import Tophat2DKernel
from regions import CircleSkyRegion, RectangleSkyRegion

from astropy.time import Time

In [2]:
#from gammapy.detect import compute_lima_on_off_image,compute_lima_image # quelle différence entre les deux ?

from gammapy.data import DataStore
from gammapy.irf import PSFKernel
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.datasets import MapDataset, Datasets
from gammapy.makers import (
    MapDatasetMaker,
    SafeMaskMaker,
    FoVBackgroundMaker,
)
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    SkyModel,
    BackgroundModel,
    PowerLawSpectralModel,
    PowerLaw2SpectralModel,
    PointSpatialModel,
    ExpCutoffPowerLawSpectralModel,
    TemplateSpatialModel,
    GaussianSpatialModel,
    FoVBackgroundModel
)
#from gammapy.stats import significance, excess # utiles ?

import gammapy
gammapy.__version__

'0.18.2'

In [3]:
src_pos = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")

In [4]:
from pathlib import Path

# Directory for outputs

path = Path("../../../hess_results/GC_variability_0.18.2/hap-fr")
path.mkdir(exist_ok=True)

pathm = Path(path/"models")
pathm.mkdir(exist_ok=True)

## Creating the geometry

In [5]:
emin, emax = [0.5, 100] * u.TeV

e_bins = 20

energy_axis = MapAxis.from_energy_bounds(
    emin.value, emax.value, e_bins, unit="TeV"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(10, 8),
    frame="galactic",
    proj="CAR",
    axes=[energy_axis],
)

geom2d = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(10, 8),
    frame="galactic",
    proj="CAR",
)

emintrue, emaxtrue = [0.3,200] * u.TeV
e_bins_true = 30

energy_axis_true = MapAxis.from_energy_bounds(
    emintrue.value, emaxtrue.value, e_bins_true, unit="TeV", name="energy_true"
)

## Model for the central source (HESS J1745-290)

In [6]:
spatial_model = PointSpatialModel(
            lon_0 = src_pos.l, 
            lat_0 = src_pos.b, 
            frame = "galactic")

spectral_model = PowerLawSpectralModel(
            #emin = 0.5* u.TeV, 
            #emax = 100* u.TeV, 
            reference = 1.0 * u.TeV,
            index = 2.3, 
            amplitude = "2.9e-12 cm-2 s-1 TeV-1")

model = SkyModel(
            spatial_model = spatial_model, 
            spectral_model = spectral_model, 
            name = "SgrA*")

model.parameters["index"].frozen = True
model.parameters["lon_0"].frozen = True
model.parameters["lat_0"].frozen = True
model.parameters["amplitude"].frozen = False
model.parameters["reference"].frozen = True # change très peu

In [7]:
spatial_model = PointSpatialModel(
            lon_0 = src_pos.l, 
            lat_0 = src_pos.b, 
            frame = "galactic")

spectral_model = ExpCutoffPowerLawSpectralModel(
        index=2, 
        amplitude=3e-12 * u.Unit("cm-2 s-1 TeV-1"), 
        reference=1 * u.TeV , 
        lambda_= 1/(10*u.TeV) , 
        alpha= 1)

modelCG_cutoff = SkyModel(
            spatial_model = spatial_model, 
            spectral_model = spectral_model, 
            name = "SgrA*")

modelCG_cutoff.parameters["index"].frozen = False
modelCG_cutoff.parameters["lambda_"].frozen = False
modelCG_cutoff.parameters["lon_0"].frozen = True
modelCG_cutoff.parameters["lat_0"].frozen = True
modelCG_cutoff.parameters["amplitude"].frozen = False
modelCG_cutoff.parameters["reference"].frozen = True

## Model for a nearby source (G09+01)

In [8]:
pos_G09 = SkyCoord(0.8791,  +0.0893, unit="deg", frame="galactic")

spatial_model2 = PointSpatialModel(
            lon_0 = pos_G09.l, 
            lat_0 = pos_G09.b, 
            frame = "galactic")

spectral_model2 = PowerLawSpectralModel(
            #emin = 0.5* u.TeV, 
            #emax = 100* u.TeV, 
            reference = 1.0 * u.TeV,
            index = 2.3, 
            amplitude = "0.88e-12 cm-2 s-1 TeV-1")

modelG09 = SkyModel(
            spatial_model = spatial_model2, 
            spectral_model = spectral_model2, 
            name = "G0901")

modelG09.parameters["index"].frozen = True
modelG09.parameters["lon_0"].frozen = True
modelG09.parameters["lat_0"].frozen = True
modelG09.parameters["amplitude"].frozen = False

## Modelling the diffuse emission 3in1

In [9]:
# GaussianSpatialModel attention pour la composante étendue et les deux sigma

CSmap = Map.read("$GAMMAPY_DATA/CS_reproj.fits") 

coords = CSmap.geom.get_coord()
lon = coords.skycoord.l.wrap_at(180*u.deg).to_value('deg')
factor = np.exp(-0.5*(lon)**2/1.11**2) #1.11 quand on a les 3 composantes
CSmap.data*=factor

CSmap.data = (CSmap.data > 10.0) * CSmap.data

solid_angle_tot = CSmap.geom.solid_angle().sum()


DGC = TemplateSpatialModel(CSmap) 

CC = GaussianSpatialModel(lon_0="0 deg", lat_0="0 deg", sigma="0.11 deg", frame='galactic') #sigma de 0.11

LSC = GaussianSpatialModel(lon_0="0 deg", lat_0="0 deg", sigma="0.97 deg", e=np.sqrt(1-(0.22/0.97)**2), phi="90 deg", frame='galactic') 
#sigma1 de 0.97 et sigma2 de 0.22, à transformer en sigma et excentricité

mapCC = CC.evaluate_geom(geom2d)*geom2d.solid_angle()
mapLSC = LSC.evaluate_geom(geom2d)*geom2d.solid_angle()
mapDGC = DGC.evaluate_geom(geom2d)*geom2d.solid_angle()

CCmax = np.sum(mapCC)
LSCmax = np.sum(mapLSC)
DGCmax = np.sum(mapDGC)

mapCC = mapCC/CCmax
mapLSC = mapLSC/LSCmax
mapDGC = mapDGC/DGCmax

data_combine = 2.68*mapLSC + 4.3*mapDGC +1.03*mapCC # facteurs pour avoir les bons ratios sur les max d'amplitude

carte = Map.from_geom(geom2d)
carte.quantity = data_combine

# in 0.15, saving a model use TemplateSpatialModel loses the base map used as a template, so we need to save it and reference it later in the associated .yaml file
carte.write(path/"models/map_DE_spatial_model", overwrite=True)

Invalid value BUNIT='Count' in FITS header. Setting empty unit.


In [10]:
DE_spatial = TemplateSpatialModel(carte) # à noter : on normalise la carte (le modèle spatial) à cette étape là


powerlaw1 =  PowerLawSpectralModel(
            reference = 1.0 * u.TeV,
            index = 2.3, 
            amplitude = "1.0e-12 cm-2 s-1 TeV-1")

model_diff = SkyModel(
            spatial_model = DE_spatial, 
            spectral_model = powerlaw1, 
            name = "DE_full")

model_diff.parameters["index"].frozen = True
#model_diff.parameters["norm"].frozen = True
model_diff.parameters["amplitude"].frozen = False
model_diff.parameters["reference"].frozen = True

model_diff.spatial_model.filename = str(pathm/"map_DE_spatial_model")

In [11]:
DE_spatial = TemplateSpatialModel(carte) # à noter : on normalise la carte (le modèle spatial) à cette étape là


spectral_model = ExpCutoffPowerLawSpectralModel(
        index=2.3, 
        amplitude=3e-12 * u.Unit("cm-2 s-1 TeV-1"), 
        reference=1 * u.TeV , 
        lambda_= 1/(10*u.TeV) , 
        alpha= 1)

model_diff_cutoff = SkyModel(
            spatial_model = DE_spatial, 
            spectral_model = spectral_model, 
            name = "DE_full")

model_diff_cutoff.parameters["index"].frozen = False
model_diff_cutoff.parameters["lambda_"].frozen = False
#model_diff_cutoff.parameters["norm"].frozen = True
model_diff_cutoff.parameters["amplitude"].frozen = False
model_diff_cutoff.parameters["reference"].frozen = True

model_diff_cutoff.spatial_model.filename = str(pathm/"map_DE_spatial_model")

## Diffuse model (3 separate components)

In [ ]:
# GaussianSpatialModel attention pour la composante étendue et les deux sigma

CSmap = Map.read("$GAMMAPY_DATA/CS_reproj.fits") 

coords = CSmap.geom.get_coord()
lon = coords.skycoord.l.wrap_at(180*u.deg).to_value('deg')
factor = np.exp(-0.5*(lon)**2/1.11**2) #1.11 quand on a les 3 composantes
CSmap.data*=factor

CSmap.data = (CSmap.data > 10.0) * CSmap.data

solid_angle_tot = CSmap.geom.solid_angle().sum()

DGC = TemplateSpatialModel(CSmap) 

In [ ]:
CC = GaussianSpatialModel(lon_0="0 deg", lat_0="0 deg", sigma="0.11 deg", frame='galactic') #sigma de 0.11

LSC = GaussianSpatialModel(lon_0="0 deg", lat_0="0 deg", sigma="0.97 deg", e=np.sqrt(1-(0.22/0.97)**2), phi="90 deg", frame='galactic') 
#sigma1 de 0.97 et sigma2 de 0.22, à transformer en sigma et excentricité


# in 0.15, saving a model use TemplateSpatialModel loses the base map used as a template, so we need to save it and reference it later in the associated .yaml file
carte.write(path/"models/map_DE_spatial_model", overwrite=True)

## Background model (post 0.18)

In [12]:
# il faut le/les datasets pour créer cet objet

## Saving the template of the model (without cutoffs)

In [13]:
models = model.copy() + modelG09.copy()+ model_diff.copy()
name = "models_template.yaml"
models.write(pathm/name, overwrite=True)

## Saving the template of the model (with cutoff for th GC)

In [14]:
models = modelCG_cutoff.copy() + modelG09.copy()+ model_diff.copy()
name = "models_template_1cutoff.yaml"
models.write(pathm/name, overwrite=True)

## Saving the template of the model (with 2 cutoffs)

In [15]:
models = modelCG_cutoff.copy() + modelG09.copy()+ model_diff_cutoff.copy()
name = "models_template_2cutoff.yaml"
models.write(pathm/name, overwrite=True)